Import required dependencies to use Google's API

In [1]:
import os.path
import pandas as pd

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

In [2]:
Credentials.from_authorized_user_info?

Signature: Credentials.from_authorized_user_info(info, scopes=None)
Docstring:
Creates a Credentials instance from parsed authorized user info.

Args:
    info (Mapping[str, str]): The authorized user info in Google
        format.
    scopes (Sequence[str]): Optional list of scopes to include in the
        credentials.

Returns:
    google.oauth2.credentials.Credentials: The constructed
        credentials.

Raises:
    ValueError: If the info is not in the expected format.
File:      c:\users\lenovo\anaconda3\envs\automation\lib\site-packages\google\oauth2\credentials.py
Type:      method

Define the accesses to be granted

In [3]:
SCOPES = ['https://www.googleapis.com/auth/classroom.courses.readonly', # View your Google Classroom classes.
          'https://www.googleapis.com/auth/classroom.rosters', # Manage your Google Classroom class rosters.
          'https://www.googleapis.com/auth/classroom.profile.emails', # View the email addresses of people in your classes.
          'https://www.googleapis.com/auth/classroom.announcements', # See, edit, and create classwork materials in Google Classroom.
          'https://www.googleapis.com/auth/spreadsheets.readonly'] # See all your Google Sheets spreadsheets.

Use the provided `token.json` file to authenticate and access user data, if `token.json` is not available, it would be created from the provided `credentials.json` by logging into user's Google Account

In [4]:
creds = None

if os.path.exists('token.json'):
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.json', 'w') as token:
        token.write(creds.to_json())

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=845305684945-mm42jq9nkk30ov1qf64ur3t855jp0e27.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A58268%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fclassroom.courses.readonly+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fclassroom.rosters+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fclassroom.profile.emails+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fclassroom.announcements+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fspreadsheets.readonly&state=Fb15acCRcJRFurOPoy6PnTxiaQw6eP&access_type=offline


In [90]:
creds = Credentials.from_authorized_user_info({"token": "ya29.a0AfB_byDHGPxtIMuoq9owPWTTZWeefjetzzPJ9wFYbf3xrsd0rxe0-bJRfmoENuHvQmgAApYeUMCugGblfG9Hl7adl3bty64Myl3WjGoDoy2xZ6_n0nrlg5WWG9qR7Uu0jOAO4E4FQViZNJDX5Trs6aa5ZK46Bjn0M9i0aCgYKAToSARESFQHGX2MiEvPnebdKkM9dfJhk0yLAfA0171", "refresh_token": "1//0gRIvT6Cv4W7CCgYIARAAGBASNwF-L9Ir5wmBpD9nqBLS-5epbD2Jwl_qBDWoMoR-ZLkgI9KWPZUy8j9yY82ioQgcWcejOZbFPUU", "token_uri": "https://oauth2.googleapis.com/token", "client_id": "845305684945-mm42jq9nkk30ov1qf64ur3t855jp0e27.apps.googleusercontent.com", "client_secret": "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", "scopes": ["https://www.googleapis.com/auth/classroom.courses.readonly", "https://www.googleapis.com/auth/classroom.rosters", "https://www.googleapis.com/auth/classroom.profile.emails", "https://www.googleapis.com/auth/classroom.announcements", "https://www.googleapis.com/auth/spreadsheets.readonly"], "expiry": "2024-01-04T09:02:05.011342Z"}, SCOPES)

In [91]:
ACTIVE_STUDENT_LINK = 'https://docs.google.com/spreadsheets/d/12FB9410fhRhZp9jl5qLe7x-LGw0QTSfLujA-dE867JE/edit#gid=1593985743'
NAMA_SHEET = 'Xion DA Night [OL]'

ACTIVE_STUDENT_ID = ACTIVE_STUDENT_LINK.split(sep='/')[-2]

Specify the sheet and the cell ranges that is going to be accessed

In [92]:
SHEET_RANGE = [f'{NAMA_SHEET}!B2:D']

Call the Google Spreadsheet API and retrieve the values of the ranges that have been specified

In [93]:
try:
    service = build('sheets', 'v4', credentials=creds)

    sheet = service.spreadsheets().values().batchGet(spreadsheetId=ACTIVE_STUDENT_ID,
                                                     ranges=SHEET_RANGE).execute()

    active_df = pd.DataFrame(sheet['valueRanges'][0]['values'], columns=['Name', 'Class', 'Email'])
    active_df['Email'] = active_df['Email'].str.strip().str.lower()
    active_df = active_df[~active_df['Name'].isin(['DV Specialization', 'ML Specialization'])]
except HttpError as error:
    print(error)

In [94]:
active_df

,Name,Class,Email
0,Alwan Husni Ramdani,[ONLINE] Data Analytics,alwanhusni@upi.edu
1,Andra Hafizha Putro,[ONLINE] Data Analytics,andrahafizha@gmail.com
2,Andryanus Agie Hendriawan,[ONLINE] Data Analytics,adhrian.agie@gmail.com
3,Arief Marzuki,[ONLINE] Academy Full Stack,arief.marzuki05@gmail.com
4,Arya Hisma Maulana,[ONLINE] Data Analytics,aryahisma@yahoo.co.id
5,Awalia Irmayanti,[ONSITE] Data Analytics,awalia.irma@gmail.com
6,Faza Aliya,[ONLINE] Academy Full Stack,faza.aliyab@gmail.com
7,Kharirotul Mizaniyah,[ONLINE] Data Analytics,mizaniyah.djalil@gmail.com
8,Lela Novi Mudi Raharti,[ONSITE] Academy Full Stack,lelanovi@gmail.com
9,Maudy Nur Avianti,[ONLINE] Academy Full Stack,maudy.n.avianti@gmail.com


In [95]:
SCORE_ACADEMY_LINK = 'https://docs.google.com/spreadsheets/d/1cGJ0pn9k9gKCBnceWVwaL9D7BBDMNjLh8uPYlaBlJi8/edit?usp=sharing' # Score Academy Link
SCORE_NAMA_SHEET = "Academy: Batch 23" # Sheet Name (Wizard) 

SCORE_ACADEMY_ID = SCORE_ACADEMY_LINK.split(sep='/')[-2]

In [96]:
'xion data analytic'.split()

['xion', 'data', 'analytic']

In [97]:
EMAIL_RANGE = [f'{SCORE_NAMA_SHEET}!D2:E']

In [98]:
try:
    service = build('sheets', 'v4', credentials=creds)

    sheet = service.spreadsheets().values().batchGet(spreadsheetId=SCORE_ACADEMY_ID,
                                                     ranges=EMAIL_RANGE).execute()

    score_df = pd.DataFrame(sheet['valueRanges'][0]['values'], columns=['Email', 'Email Classroom'])
    score_df['Email'] = score_df['Email'].str.strip().str.lower()
    score_df['Email Classroom'] = score_df['Email Classroom'].str.strip().str.lower()
    
except HttpError as error:
    print(error)

In [99]:
score_df

,Email,Email Classroom
0,danadyaksawijaya@gmail.com,danadyaksawijaya@gmail.com
1,berlianahar@gmail.com,berlianahar@gmail.com
2,angiegrace35@gmail.com,angiegrace35@gmail.com
3,ihsan.azmi.rabbani@gmail.com,ihsan.azmi.rabbani@gmail.com
4,lhpratama2@gmail.com,lhpratama2@gmail.com
5,melissa.offc@gmail.com,melissa.offc@gmail.com
6,nadyanaftalia@gmail.com,nadyanaftalia@gmail.com
7,brenda.thadete@live.com,brenda.thadete@gmail.com
8,ghozyaufan@gmail.com,ghozyaufan@gmail.com
9,kambu.charles@aol.com,kambu.charles@gmail.com


In [107]:
classroom_df = pd.merge(active_df, score_df, how='left', on='Email')
classroom_df = classroom_df[classroom_df['Email'].notna()]
classroom_df

,Name,Class,Email,Email Classroom
0,Alwan Husni Ramdani,[ONLINE] Data Analytics,alwanhusni@upi.edu,NaN
1,Andra Hafizha Putro,[ONLINE] Data Analytics,andrahafizha@gmail.com,NaN
2,Andryanus Agie Hendriawan,[ONLINE] Data Analytics,adhrian.agie@gmail.com,NaN
3,Arief Marzuki,[ONLINE] Academy Full Stack,arief.marzuki05@gmail.com,NaN
4,Arya Hisma Maulana,[ONLINE] Data Analytics,aryahisma@yahoo.co.id,NaN
5,Awalia Irmayanti,[ONSITE] Data Analytics,awalia.irma@gmail.com,NaN
6,Faza Aliya,[ONLINE] Academy Full Stack,faza.aliyab@gmail.com,faza.aliyab@gmail.com
7,Kharirotul Mizaniyah,[ONLINE] Data Analytics,mizaniyah.djalil@gmail.com,NaN
8,Lela Novi Mudi Raharti,[ONSITE] Academy Full Stack,lelanovi@gmail.com,lelanovi@gmail.com
9,Maudy Nur Avianti,[ONLINE] Academy Full Stack,maudy.n.avianti@gmail.com,NaN


In [110]:
classroom_df.loc[classroom_df['Email Classroom'].notna(), 'Status'] = 'Assigned ✅'
classroom_df.loc[classroom_df['Email Classroom'].isna(), 'Status'] = 'Missing  ❌'
classroom_df

,Name,Class,Email,Email Classroom,Status
0,Alwan Husni Ramdani,[ONLINE] Data Analytics,alwanhusni@upi.edu,NaN,Missing ❌
1,Andra Hafizha Putro,[ONLINE] Data Analytics,andrahafizha@gmail.com,NaN,Missing ❌
2,Andryanus Agie Hendriawan,[ONLINE] Data Analytics,adhrian.agie@gmail.com,NaN,Missing ❌
3,Arief Marzuki,[ONLINE] Academy Full Stack,arief.marzuki05@gmail.com,NaN,Missing ❌
4,Arya Hisma Maulana,[ONLINE] Data Analytics,aryahisma@yahoo.co.id,NaN,Missing ❌
5,Awalia Irmayanti,[ONSITE] Data Analytics,awalia.irma@gmail.com,NaN,Missing ❌
6,Faza Aliya,[ONLINE] Academy Full Stack,faza.aliyab@gmail.com,faza.aliyab@gmail.com,Assigned ✅
7,Kharirotul Mizaniyah,[ONLINE] Data Analytics,mizaniyah.djalil@gmail.com,NaN,Missing ❌
8,Lela Novi Mudi Raharti,[ONSITE] Academy Full Stack,lelanovi@gmail.com,lelanovi@gmail.com,Assigned ✅
9,Maudy Nur Avianti,[ONLINE] Academy Full Stack,maudy.n.avianti@gmail.com,NaN,Missing ❌


In [16]:
service = build('classroom', 'v1', credentials=creds)

In [17]:
results = service.courses().list(pageSize=20).execute()
courses = results.get('courses', [])

if not courses:
    print('No courses found.')

# Prints the names of the first 5 courses.
print('Courses:')
for course in courses:
    print(f"- {course['name']}")

Courses:
- HSBC - Data Science Batch 2
- HSBC - Data Science Batch 1
- XL - Development Program
- XL - Development Program
- Dummy Machine Learning
- Xion Data Visualization
- Dummy Xion DA
- Xion Data Analytics
- Copy of Wizard Data Visualization
- Wizard Data Analytics
- PAXEL - Data Analytics
- Data Science for Everyone
- DSS: Data Science Fundamentals
- DSS: Geospatial Data Visualization with Rayshader In R
- Wizard Machine Learning
- Wizard Data Visualization
- Kementerian BUMN
- Automation
- Wizard Data Analytics
- Vulcan Machine Learning


In [18]:
course_input = input("Enter Google Classroom Class Name : ")
course_lowercase = course_input.lower()
course_id = None

for course in courses:
    if course_lowercase == course['name'].lower():
        course_id = course['id']
        break

if course_id == None:
    raise Exception(f"{course_input} course not found")

else:
    print(f'{course_input} found with ID {course_id}')

Xion Data Analytics found with ID 530430758394


In [19]:
response = service.courses().students().list(courseId=course_id).execute()
students = response.get('students')

while response.get('nextPageToken'):
    response = service.courses().students().list(courseId=course_id, pageToken = response['nextPageToken']).execute()
    students.extend(response.get('students'))

In [20]:
student_id = []

for student in students:
    if student['profile']['emailAddress'] in classroom_df['Email Classroom'].tolist():
        student_id.append(student['userId'])
    elif "@algorit.ma" in student['profile']['emailAddress']:
        student_id.append(student['userId'])

In [21]:
student_id

['110296152368851563506',
 '117751670129098765183',
 '115675990890822598299',
 '115277787894232873138',
 '102125295425146264827',
 '106619458262770403723',
 '113810785755953630319',
 '117015102013261065396',
 '103789689146955498153',
 '118184562394375302234',
 '105017933107430151332',
 '103406729334638391553',
 '112917448702615862231',
 '109614082986259498644',
 '109257367130303493443',
 '113242723966671793858',
 '111777795762986619924',
 '116085810387530002905',
 '108931938371256497403']

In [22]:
body = {
    'text': "testing draft 3",
    'state': "DRAFT",
    'assigneeMode': 'INDIVIDUAL_STUDENTS',
    "individualStudentsOptions": {
    "studentIds": student_id
  }
}
response = service.courses().announcements().create(courseId=course_id, body=body).execute()